# **Visual Weather Analysis** from Location Freiburg in Germany

---
### Imports

In [105]:
import numpy as np
import pandas as pd

import plotly_express as px

---
### Data Loading & Wrangling

In [106]:
df = pd.read_csv("../data/weather/Freiburg_im_Breisgau_Wetter.csv")
old_rows = df.shape[0]
df.head(2)

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,946684800,2000-01-01 00:00:00 +0000 UTC,3600,Freiburg im Breisgau,47.999008,7.842104,1.54,NaN,0.68,-0.10,...,NaN,NaN,NaN,NaN,NaN,93,804,Clouds,overcast clouds,04n
1,946688400,2000-01-01 01:00:00 +0000 UTC,3600,Freiburg im Breisgau,47.999008,7.842104,1.64,NaN,0.78,1.64,...,NaN,NaN,NaN,NaN,NaN,95,804,Clouds,overcast clouds,04n


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194326 entries, 0 to 194325
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt                   194326 non-null  int64  
 1   dt_iso               194326 non-null  object 
 2   timezone             194326 non-null  int64  
 3   city_name            194326 non-null  object 
 4   lat                  194326 non-null  float64
 5   lon                  194326 non-null  float64
 6   temp                 194326 non-null  float64
 7   visibility           1174 non-null    float64
 8   dew_point            194326 non-null  float64
 9   feels_like           194326 non-null  float64
 10  temp_min             194326 non-null  float64
 11  temp_max             194326 non-null  float64
 12  pressure             194326 non-null  int64  
 13  sea_level            0 non-null       float64
 14  grnd_level           0 non-null       float64
 15  humidity         

In [108]:
df['visibility'].unique()

array([   nan,  9000., 10000.,   100.,  8000.,  6000.,  7000.,   600.,
         500.,  4500.,   200.,   400.,  5000.,  2500.])

In [109]:
df['visibility'].value_counts()

9000.0     965
10000.0    183
100.0        6
8000.0       6
6000.0       4
7000.0       3
600.0        1
500.0        1
4500.0       1
200.0        1
400.0        1
5000.0       1
2500.0       1
Name: visibility, dtype: int64

drop columns with to many nan values

In [110]:
df.isna().sum()

dt                          0
dt_iso                      0
timezone                    0
city_name                   0
lat                         0
lon                         0
temp                        0
visibility             193152
dew_point                   0
feels_like                  0
temp_min                    0
temp_max                    0
pressure                    0
sea_level              194326
grnd_level             194326
humidity                    0
wind_speed                  0
wind_deg                    0
wind_gust              174306
rain_1h                150740
rain_3h                194295
snow_1h                191781
snow_3h                194326
clouds_all                  0
weather_id                  0
weather_main                0
weather_description         0
weather_icon                0
dtype: int64

In [111]:
df = df.drop(columns=['visibility', 'sea_level', 'grnd_level', 'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h'])

In [112]:
df.isna().sum()

dt                     0
dt_iso                 0
timezone               0
city_name              0
lat                    0
lon                    0
temp                   0
dew_point              0
feels_like             0
temp_min               0
temp_max               0
pressure               0
humidity               0
wind_speed             0
wind_deg               0
clouds_all             0
weather_id             0
weather_main           0
weather_description    0
weather_icon           0
dtype: int64

drop / change rest of columns

In [113]:
df.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'dew_point', 'feels_like', 'temp_min', 'temp_max', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')

In [114]:
df.drop(columns=['dt', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon', 'temp_min', 'temp_max', 'weather-id'], inplace=True)

In [115]:
# remove UTC by date
df['dt_iso'] = df['dt_iso'].replace("\s*\+.*UTC", "", regex=True)
df.loc[:3, ['dt_iso']]

,dt_iso
0,2000-01-01 00:00:00
1,2000-01-01 01:00:00
2,2000-01-01 02:00:00
3,2000-01-01 03:00:00


create data time and set to index

In [116]:
#df[['date', 'time']] = df['dt_iso'].str.split(" ", 1, expand=True)
#df.drop(columns=['dt_iso'], inplace=True)
#df['time'] = pd.to_datetime(df['time'])
#df['date'] = pd.to_datetime(df['date'])

df.rename(columns={'dt_iso':'dt'}, inplace=True)
df['dt'] = pd.to_datetime(df['dt'])
df = df.set_index('dt', drop=False)

In [117]:
df.head()

,dt,temp,dew_point,feels_like,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description
dt,,,,,,,,,,,,
2000-01-01 00:00:00,2000-01-01 00:00:00,1.54,0.68,-0.10,1026,94,1.57,168,93,804,Clouds,overcast clouds
2000-01-01 01:00:00,2000-01-01 01:00:00,1.64,0.78,1.64,1026,94,1.21,163,95,804,Clouds,overcast clouds
2000-01-01 02:00:00,2000-01-01 02:00:00,1.14,0.28,-0.62,1026,94,1.61,163,95,804,Clouds,overcast clouds
2000-01-01 03:00:00,2000-01-01 03:00:00,1.24,0.53,-0.57,1026,95,1.66,163,95,804,Clouds,overcast clouds
2000-01-01 04:00:00,2000-01-01 04:00:00,1.61,0.90,0.07,1026,95,1.51,159,99,804,Clouds,overcast clouds


downsampling

In [149]:
# downsamlping
# df.resample('2H').sum()

freq = '1D'

df_per_day = None
to_do = [('temp', True, ['min', 'max', 'mean', 'median']),
         ('dew_point', True, ['min', 'max', 'mean', 'median']),
         ('feels_like', True, ['min', 'max', 'mean', 'median']),
         ('pressure', True, ['min', 'max', 'mean', 'median']),
         ('humidity', True, ['min', 'max', 'mean', 'median']),
         ('wind_speed', True, ['min', 'max', 'mean', 'median']),
         ('wind_deg', True, ['min', 'max', 'mean', 'median']),
         ('clouds_all', True, ['min', 'max', 'mean', 'median']),
         ('weather_description', False, np.max),
         ('weather_main', False, np.max)]
for col_name, should_use_agg, funcs in to_do:
    if should_use_agg:
        new_df = df[col_name].resample(freq).agg(funcs).rename(columns={i:col_name+"_"+i for i in funcs}, inplace=False)
    else:
        new_df = df[col_name].groupby(pd.Grouper(freq=freq)).apply(funcs)

    if type(df_per_day) != pd.Series and type(df_per_day) != pd.DataFrame and df_per_day == None:
        df_per_day = new_df
    else:
        df_per_day = df_per_day.join(new_df, how='outer')


In [150]:
df_per_day.head()

,temp_min,temp_max,temp_mean,temp_median,dew_point_min,dew_point_max,dew_point_mean,dew_point_median,feels_like_min,feels_like_max,...,wind_deg_min,wind_deg_max,wind_deg_mean,wind_deg_median,clouds_all_min,clouds_all_max,clouds_all_mean,clouds_all_median,weather_description,weather_main
dt,,,,,,,,,,,,,,,,,,,,,
2000-01-01,1.14,4.98,3.060833,3.580,0.28,4.25,2.370000,2.90,-0.62,3.84,...,158,226,192.000000,199.5,92,100,98.458333,100.0,overcast clouds,Rain
2000-01-02,1.81,7.11,3.481250,2.810,0.45,6.36,2.672500,2.31,-0.16,7.11,...,79,189,120.583333,115.0,2,100,62.375000,82.0,sky is clear,Rain
2000-01-03,-1.72,3.64,1.550000,2.150,-2.98,1.82,0.105417,0.73,-4.63,3.64,...,135,188,152.250000,150.5,3,75,33.375000,36.5,sky is clear,Clouds
2000-01-04,-1.56,9.33,5.344167,6.655,-2.95,5.77,3.078333,4.70,-4.30,7.91,...,160,220,186.458333,181.0,5,100,59.750000,60.5,sky is clear,Rain
2000-01-05,1.09,11.04,6.444583,6.535,0.67,9.95,5.553333,5.53,-1.78,10.63,...,114,232,176.958333,179.0,52,100,90.958333,100.0,overcast clouds,Rain


---
### Start Exploring

In [155]:
fig = px.line(df_per_day, x=df_per_day.index, y='temp_mean', title="Mean Temperature")
fig.show()

In [165]:
fig = px.line(df_per_day, x=df_per_day.index, y='temp_mean', title="Mean Temperature and Humidity")
fig.add_scatter(x=df_per_day.index, y=df_per_day['humidity_mean'], mode='lines', cliponaxis=False)
fig.show()

In [159]:
fig = px.scatter(df_per_day, x=df_per_day.index, y='temp_mean', template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10,
                 trendline = 'ols')
fig.update_traces(mode = 'lines')
fig.data[-1].line.color = 'red'
fig.show()